In [ ]:
#### jefan 
#### begun: Dec 29 2016, updated: Jul 15 2018
#### analysis pipeline for "Results: Consequences for object recognition"

In [ ]:
import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
np = numpy
plt = pyplot
from __future__ import division
import pandas as pd
import json
import os, sys

from IPython.core.pylabtools import figsize, getfigs

%matplotlib inline
from pylab import *
from numpy import *
import scipy.stats as stats
import seaborn as sns
import prettyplotlib as ppl

import warnings
warnings.filterwarnings('ignore')

In [ ]:
## model-fitting helpers
from scipy.optimize import curve_fit

def sigmoid(x, x0, k):
    y = 1 / (1 + np.exp(-k*(x-x0)))
    return y 
    
def sigmoid_alt(x,x0,k,L):
    y = L / (1 + np.exp(-k*(x-x0)))
    return y

def sigmoid_alt4(x,x0,k,L,G):
    y = G + ((1 - G - L) / (1 + np.exp(-k*(x-x0))))

def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])

def sigmoid_derivative(x, x0, k):
    f = np.exp(-k*(x-x0))
    return -k / f

def fit_sigmoid(x,y):
    Ns = len(x)
    threshold = []
    slope = []
    for ss in range(Ns):          
        xdata = x[ss]
        ydata = y[ss]
        skip = 0        
        try:
            popt, pcov = curve_fit(sigmoid, xdata, ydata,maxfev=300)
            if (popt[0]>0) & (popt[0]<1):
                threshold.append(popt[0])
            else:
                threshold.append(nan)
            slope.append(popt[1])                           
        except:                     
            try:
                popt, pcov = curve_fit(sigmoid_alt, xdata, ydata,maxfev=800)
                if (popt[0]>0) & (popt[0]<1):
                    threshold.append(popt[0])
                else:
                    threshold.append(nan)
                slope.append(popt[1])                
            except:
                plt.figure(figsize(2,2))
                skip = 1
                plt.plot(xdata,ydata) ## plot the bad fit
                plt.title('failed fits')
                print 'subject: ' + workers_repd[ss] + ' version: ' + str(versions_repd[ss])
                threshold.append(nan)
                slope.append(nan)            
    return map(np.array,[threshold,slope])

def remove_nans(x):
    return x[~np.isnan(x)]

def compare_phases(pre,post):
    diff = post-pre
    prop = (post-pre)/pre
    return diff,prop


#### set up paths

In [ ]:
## define data directory (where you put downloaded data)
data_dir = './data'

## load in pre-post data matrix for 
CPD = pd.read_csv(os.path.join(data_dir,'Categorical_Perception_Recognition_Data_Drawing.csv'))
CPO = pd.read_csv(os.path.join(data_dir,'Categorical_Perception_Recognition_Data_Observation.csv'))

## Analyze pre-post learning from main experiment (Drawing cohort)

In [ ]:
###### extract key variables from main experiment (drawing cohort)
pp = CPD
numSubs = len(np.unique(CPD['wID']))
p_o1_trained_pre = np.array(pp[(pp['cond']=='trained') & (pp['phase']=='pre')]['p_o1']).reshape(numSubs,6)
p_o1_trained_post = np.array(pp[(pp['cond']=='trained') & (pp['phase']=='post')]['p_o1']).reshape(numSubs,6)
p_o1_near_pre = np.array(pp[(pp['cond']=='near') & (pp['phase']=='pre')]['p_o1']).reshape(numSubs,6)
p_o1_near_post = np.array(pp[(pp['cond']=='near') & (pp['phase']=='post')]['p_o1']).reshape(numSubs,6)
baseprop_trained_pre = np.array(pp[(pp['cond']=='trained') & (pp['phase']=='pre')]['baseprop']).reshape(numSubs,6)
baseprop_trained_post = np.array(pp[(pp['cond']=='trained') & (pp['phase']=='post')]['baseprop']).reshape(numSubs,6)
baseprop_near_pre = np.array(pp[(pp['cond']=='near') & (pp['phase']=='pre')]['baseprop']).reshape(numSubs,6)
baseprop_near_post = np.array(pp[(pp['cond']=='near') & (pp['phase']=='post')]['baseprop']).reshape(numSubs,6)

In [ ]:
## fit logistic function to psychometric data, get slope and threshold parameters
threshold_trained_pre,slope_trained_pre = fit_sigmoid(baseprop_trained_pre,p_o1_trained_pre)
threshold_trained_post,slope_trained_post = fit_sigmoid(baseprop_trained_post,p_o1_trained_post)
threshold_near_pre,slope_near_pre = fit_sigmoid(baseprop_near_pre,p_o1_near_pre)
threshold_near_post,slope_near_post = fit_sigmoid(baseprop_near_post,p_o1_near_post)

## calculate change in slope
diff_trained,prop_trained = compare_phases(slope_trained_pre,slope_trained_post)
diff_near,prop_near = compare_phases(slope_near_pre,slope_near_post)

slope_trained_diff = slope_trained_post-slope_trained_pre
slope_near_diff = slope_near_post-slope_near_pre

## calculate change in threshold
diff_thresh_trained,prop_thresh_trained = compare_phases(threshold_trained_pre,threshold_trained_post)
diff_thresh_near,prop_thresh_near = compare_phases(threshold_near_pre,threshold_near_post)


In [ ]:
def tabulate_signs(x,rowlabel):
    x = remove_nans(x)
    print rowlabel + ': <0: ' + str(np.round(sum(x<0)/len(x),4)) + '  =0: ' + str(np.round(sum(x==0)/len(x),4)) + '  >0: ' + str(np.round(sum(x>0)/len(x),4))       

print 'Drawing Cohort'
print 'PROPORTION OF SUBJECTS WITH SHALLOWER SLOPES (<0), EQUAL SLOPES (=0), OR STEEPER SLOPES (>0)'    
tabulate_signs(diff_trained,'trained')
tabulate_signs(diff_near,'near')

In [ ]:
def get_boot_CI(data,nIter):
    boot_mean = []
    for n in range(nIter):
        bootgroup = np.random.RandomState(n).choice(data,size=len(data),replace=True) 
        boot_mean.append(np.nanmean(bootgroup))
    boot_mean = map(np.array,[boot_mean]) 
    return np.mean(boot_mean),np.percentile(boot_mean,2.5), np.percentile(boot_mean,97.5)

def get_boot(data,nIter):
    boot_mean = []
    for n in range(nIter):
        bootgroup = np.random.RandomState(n).choice(data,size=len(data),replace=True) 
        boot_mean.append(np.nanmean(bootgroup))
    boot_mean = np.array(boot_mean)
    return boot_mean

def get_boot_pval(data,nIter):
    boot_mean = get_boot(data,nIter)
    if np.mean(boot_mean) > 0:
        p = (sum(boot_mean<0)/len(boot_mean)) * 2
    elif np.mean(boot_mean) < 0:
        p = (sum(boot_mean>0)/len(boot_mean)) * 2
    else:
        p = (sum(boot_mean<0)/len(boot_mean)) * 2
    return p

def get_boot_SEM(data,nIter):
    boot_mean = []
    for n in range(nIter):
        bootgroup = np.random.RandomState(n).choice(data,size=len(data),replace=True) 
        boot_mean.append(np.nanmean(bootgroup))
    boot_mean = map(np.array,[boot_mean]) 
    return np.mean(boot_mean),np.percentile(boot_mean,16), np.percentile(boot_mean,84)

In [ ]:
## Plotting quantile-quantile plot against normal distribution 
def normalize(x):
    mu = np.mean(x)
    sd = np.std(x)
    return (x-mu)/sd

sns.set_context('talk')
fig = plt.figure(figsize=(9,3))
plt.subplot(1,2,1)
data = normalize(slope_trained_pre-slope_trained_post)
res = stats.probplot(data,plot=plt)
plt.subplot(1,2,2)
data = normalize(slope_near_pre-slope_near_post)
res = stats.probplot(data,plot=plt)

## Quantile-quantile plots reveal departure from normal distribution. 
## Proceeded with nonparametric estimation and inference procedures.

In [ ]:
## test for difference at pre-test
nIter = 10000
print 'Drawing Cohort: Slope'
print '###### pretest: trained vs. near ######'
data = slope_trained_pre-slope_near_pre
p = get_boot_pval(data,nIter)
print p

In [ ]:
## test for reliability of change in slope (bootstrapped sampling distribution of mean b/c non-normal distribution)
nIter = 10000
print 'Drawing Cohort: Slope'
print '###### trained pre to post ######'
data = slope_trained_pre-slope_trained_post
p = get_boot_pval(data,nIter)
print 'p = ', p
print '###### near pre to post ######'
data = slope_near_pre-slope_near_post
p = get_boot_pval(data,nIter)
print 'p = ', p
print '###### interaction between trained and near ######'
data = diff_trained-diff_near
p = get_boot_pval(data,nIter)
print 'p = ', p

In [ ]:
## test for reliability of change in threshold (bootstrapped sampling distribution of mean b/c non-normal distribution)
nIter = 10000
print 'Drawing Cohort: Threshold'
print '###### trained pre to post ######'
data = threshold_trained_pre-threshold_trained_post
p = get_boot_pval(data,nIter)
print 'p = ', p
print '###### near pre to post ######'
data = threshold_near_pre-threshold_near_post
p = get_boot_pval(data,nIter)
print 'p = ', p


In [ ]:
## plot change in slope for main experiment (drawing cohort)
STDD = slope_trained_diff
SNDD = slope_near_diff
condition_colors = [(0.7,0.4,0.2),(0.66,0.66,0.66)]
sns.set_style('white')
sns.set_context('poster')
fig = plt.figure(figsize=(4,4))
m = map(np.nanmean,[STDD, SNDD])
mt,lbt,ubt = get_boot_SEM(STDD,5000)
mn,lbn,ubn = get_boot_SEM(SNDD,5000)
plt.ylim([-30,40])
ppl.bar(np.arange(0.1,2.1),m,yerr=np.array([[lbt-mt,ubt-mt],[lbn-mn,ubn-mn]]), \
        ecolor=[0.3,0.3,0.3],color=condition_colors, width=0.8,xticklabels=['Trained','Control'])
plt.ylabel('Change in slope (post-pre)')
plt.title('Drawing Cohort')
plt.savefig('./plots/3_prepost_slope_drawing_cohort.pdf')
plt.tight_layout()
plt.close(fig)

## Control Experiment (Dynamic Observation cohort)

In [ ]:
###### extract key variables from control experiment (Observation cohort)
pp = CPO
numSubs = len(np.unique(CPD['wID']))
p_o1_trained_pre = np.array(pp[(pp['cond']=='trained') & (pp['phase']=='pre')]['p_o1']).reshape(numSubs,6)
p_o1_trained_post = np.array(pp[(pp['cond']=='trained') & (pp['phase']=='post')]['p_o1']).reshape(numSubs,6)
p_o1_near_pre = np.array(pp[(pp['cond']=='near') & (pp['phase']=='pre')]['p_o1']).reshape(numSubs,6)
p_o1_near_post = np.array(pp[(pp['cond']=='near') & (pp['phase']=='post')]['p_o1']).reshape(numSubs,6)
baseprop_trained_pre = np.array(pp[(pp['cond']=='trained') & (pp['phase']=='pre')]['baseprop']).reshape(numSubs,6)
baseprop_trained_post = np.array(pp[(pp['cond']=='trained') & (pp['phase']=='post')]['baseprop']).reshape(numSubs,6)
baseprop_near_pre = np.array(pp[(pp['cond']=='near') & (pp['phase']=='pre')]['baseprop']).reshape(numSubs,6)
baseprop_near_post = np.array(pp[(pp['cond']=='near') & (pp['phase']=='post')]['baseprop']).reshape(numSubs,6)

In [ ]:
## fit logistic function to psychometric data, get slope and threshold parameters
threshold_trained_pre,slope_trained_pre = fit_sigmoid(baseprop_trained_pre,p_o1_trained_pre)
threshold_trained_post,slope_trained_post = fit_sigmoid(baseprop_trained_post,p_o1_trained_post)
threshold_near_pre,slope_near_pre = fit_sigmoid(baseprop_near_pre,p_o1_near_pre)
threshold_near_post,slope_near_post = fit_sigmoid(baseprop_near_post,p_o1_near_post)

## calculate change in slope
diff_trained,prop_trained = compare_phases(slope_trained_pre,slope_trained_post)
diff_near,prop_near = compare_phases(slope_near_pre,slope_near_post)

slope_trained_diff = slope_trained_post-slope_trained_pre
slope_near_diff = slope_near_post-slope_near_pre

## calculate change in threshold
diff_thresh_trained,prop_thresh_trained = compare_phases(threshold_trained_pre,threshold_trained_post)
diff_thresh_near,prop_thresh_near = compare_phases(threshold_near_pre,threshold_near_post)


In [ ]:
def tabulate_signs(x,rowlabel):
    x = remove_nans(x)
    print rowlabel + ': <0: ' + str(np.round(sum(x<0)/len(x),4)) + '  =0: ' + str(np.round(sum(x==0)/len(x),4)) + '  >0: ' + str(np.round(sum(x>0)/len(x),4))       

print 'Observation Cohort'
print 'PROPORTION OF SUBJECTS WITH SHALLOWER SLOPES (<0), EQUAL SLOPES (=0), OR STEEPER SLOPES (>0)'    
tabulate_signs(diff_trained,'trained')
tabulate_signs(diff_near,'near')

In [ ]:
## Plotting quantile-quantile plot against normal distribution 
def normalize(x):
    mu = np.mean(x)
    sd = np.std(x)
    return (x-mu)/sd

sns.set_context('talk')
fig = plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
data = normalize(slope_trained_pre-slope_trained_post)
res = stats.probplot(data,plot=plt)
plt.subplot(1,2,2)
data = normalize(slope_near_pre-slope_near_post)
res = stats.probplot(data,plot=plt)

## Quantile-quantile plots reveal departure from normal distribution. 
## Proceeded with nonparametric estimation and inference procedures.

In [ ]:
## test for reliability of change in slope (bootstrapped sampling distribution of mean b/c non-normal distribution)
nIter = 10000
print 'Drawing Cohort'
print '###### trained pre to post ######'
data = slope_trained_pre-slope_trained_post
p = get_boot_pval(data,nIter)
print 'p = ', p
print '###### near pre to post ######'
data = slope_near_pre-slope_near_post
p = get_boot_pval(data,nIter)
print 'p = ', p
print '###### interaction between trained and near ######'
data = diff_trained-diff_near
p = get_boot_pval(data,nIter)
print 'p = ', p

In [ ]:
## plot change in slope for control experiment (Observation cohort)
STDO = slope_trained_diff
SNDO = slope_near_diff
condition_colors = [(0.7,0.4,0.2),(0.66,0.66,0.66)]
sns.set_style('white')
sns.set_context('poster')
fig = plt.figure(figsize=(4,4))
m = map(np.nanmean,[STDO, SNDO])
mt,lbt,ubt = get_boot_SEM(STDO,5000)
mn,lbn,ubn = get_boot_SEM(SNDO,5000)
plt.ylim([-30,40])
ppl.bar(np.arange(0.1,2.1),m,yerr=np.array([[lbt-mt,ubt-mt],[lbn-mn,ubn-mn]]), \
        ecolor=[0.3,0.3,0.3],color=condition_colors, width=0.8,xticklabels=['Trained','Control'])
plt.ylabel('Change in slope (post-pre)')
plt.title('Observation Cohort')
plt.savefig('./plots/3_prepost_slope_observation_cohort.pdf')
plt.tight_layout()
plt.close(fig)

In [ ]:
## test for interaction between group (Drawing vs. Observation) and condition (Trained vs. Control)
nIter = 10000
print 'Interaction between group (Drawing vs. Observation) and condition (Trained vs. Control)'
data = (STDD-SNDD)-(STDO-SNDO)
p = get_boot_pval(data,nIter)
print 'p = ', p

In [ ]:
## test for reliability of change in threshold (bootstrapped sampling distribution of mean b/c non-normal distribution)
nIter = 10000
print 'Observation Cohort: Threshold'
print '###### trained pre to post ######'
data = threshold_trained_pre-threshold_trained_post
p = get_boot_pval(data,nIter)
print 'p = ', p
print '###### near pre to post ######'
data = threshold_near_pre-threshold_near_post
p = get_boot_pval(data,nIter)
print 'p = ', p
